# 百度搜索通用问答机器人

## 目标
使用百度搜索，无需自建知识库，对用户的请求进行回答。

用户可通过自定义人设来创建自己的问答机器人服务，并可通过开关配置拒答、澄清反问、重点强调、友好度提升、溯源等能力。

## 准备工作
### 平台注册

1.先在appbuilder平台注册，获取token

2.安装appbuilder-sdk

In [ ]:
!pip install appbuilder-sdk

In [ ]:
import os

#  设置环境变量
os.environ["APPBUILDER_TOKEN"] = "..."

print("init done")


## 开发过程
### 配置 RAGWithBaiduSearch 组件

In [ ]:
import appbuilder

# 人设指令
instruction = appbuilder.Message("你是问答助手，在回答问题前需要加上“很高兴为您解答“")

# 能力开关
reject = False # 拒答
clarify = False # 澄清反问
highlight = True # 重点强调
friendly = False # 友好度提升
cite = True # 溯源

# 使用 Qianfan-Agent-Speed-8K 模型
component = appbuilder.RAGWithBaiduSearch(model="Qianfan-Agent-Speed-8K")
query = appbuilder.Message("海淀区的面积是多少")

answer = component.run(
    query, instruction=instruction, reject=reject, clarify=clarify, highlight=highlight, friendly=friendly, cite=cite, stream=False)

print(f"问答结果: {answer.content}")
print(f"检索返回: {answer.extra}")

### (可选) 使用 ChainlitRuntime 启动 chainlit 页面调试
这部分代码依赖 `appbuilder-sdk[serve]`，如果没有安装，可以执行下面的命令安装：

In [ ]:
!pip install 'appbuilder-sdk[serve]'

执行下面的代码，会启动一个 chainlit 页面，页面地址：0.0.0.0:8091

In [ ]:
import appbuidler
from appbuilder.utils.chainlit_deploy import ChainlitRuntime
agent = ChainlitRuntime(component=component)
# 启动 chainlit 服务
agent.chainlit_component(port=8091)

### (可选) 使用 FlaskRuntime 启动 HTTP 服务
这部分代码依赖 `appbuilder-sdk[serve]`，如果没有安装，可以执行下面的命令安装：

In [ ]:
!pip install 'appbuilder-sdk[serve]'

执行下面的代码，会启动一个 HTTP 服务

In [ ]:
import appbuilder
from appbuilder.utils.flask_deploy import FlaskRuntime
agent = FlaskRuntime(component=component)
# 启动 flask 服务
agent.serve(port=8092)

测试服务

In [ ]:
curl --location 'http://0.0.0.0:8092/chat' \
--header 'Content-Type: application/json' \
--data '{
    "message": "海淀区的面积是多少",
    "stream": false
}'